In [9]:
import torch
import math
from torch import nn
import torch.nn.functional as F

In [10]:
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1] #64
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k) # 30*50*50
    print(f"scaled.size() : {scaled.size()}")
    if mask is not None:
        print(f"-- ADDING MASK of shape {mask.size()} --") 
        # Broadcasting add. So just the last N dimensions need to match
        scaled += mask
    attention = F.softmax(scaled, dim=-1)   # 30*50*50
    values = torch.matmul(attention, v)    # 30*50*64
    return values, attention

In [11]:
class MultiHeadAttention(nn.Module):

    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model    # 512
        self.num_heads = num_heads #8
        self.head_dim = d_model // num_heads # 64
        self.qkv_layer = nn.Linear(d_model , 3 * d_model)    # 512*1536
        self.linear_layer = nn.Linear(d_model, d_model)      # 512*512
    
    def forward(self, x, mask=None):
        batch_size, max_sequence_length, d_model = x.size()  # 30 , 50 ,512
        print(f"x.size(): {x.size()}")
        qkv = self.qkv_layer(x)              #30*50*1536
        print(f"qkv.size(): {qkv.size()}")
        qkv = qkv.reshape(batch_size, max_sequence_length, self.num_heads, 3 * self.head_dim)   #30*50*8*192
        print(f"qkv.size(): {qkv.size()}")
        qkv = qkv.permute(0, 2, 1, 3)    #30*8*50*192
        print(f"qkv.size(): {qkv.size()}")
        q, k, v = qkv.chunk(3, dim=-1)    #30*50*8*64
        print(f"q size: {q.size()}, k size: {k.size()}, v size: {v.size()}, ")
        values, attention = scaled_dot_product(q, k, v, mask)   # values = 30*50*8*64 , attention = 30*50*50
        print(f"values.size(): {values.size()}, attention.size:{ attention.size()} ")
        values = values.reshape(batch_size, max_sequence_length, self.num_heads * self.head_dim)  #30*50*512
        print(f"values.size(): {values.size()}")
        out = self.linear_layer(values)  #30*50*512
        print(f"out.size(): {out.size()}")
        return out

In [12]:
class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape   # [512]      i think last dim = 1
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))  #  [512]      512*1
        self.beta =  nn.Parameter(torch.zeros(parameters_shape))  #  [512]      512*1

    def forward(self, inputs):   #30*50*512
        dims = [-(i + 1) for i in range(len(self.parameters_shape))]  #[-1]
        mean = inputs.mean(dim=dims, keepdim=True)   #30*50*1
        print(f"Mean ({mean.size()})")
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)  #30*50*1
        std = (var + self.eps).sqrt()            #30*50*1
        print(f"Standard Deviation  ({std.size()})")
        y = (inputs - mean) / std     # 30*50*512
        print(f"y: {y.size()}")
        out = self.gamma * y  + self.beta      # 30*50*512
        print(f"self.gamma: {self.gamma.size()}, self.beta: {self.beta.size()}")
        print(f"out: {out.size()}")
        return out

In [13]:
class PositionwiseFeedForward(nn.Module):

    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)     #512*2048
        self.linear2 = nn.Linear(hidden, d_model)   #2048*512
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):    #30*50*512
        x = self.linear1(x)  #30*50*2048
        print(f"x after first linear layer: {x.size()}")
        x = self.relu(x)     #30*50*2048
        print(f"x after activation: {x.size()}")
        x = self.dropout(x)     #30*50*2048
        print(f"x after dropout: {x.size()}")
        x = self.linear2(x)       # 30*50*512
        print(f"x after 2nd linear layer: {x.size()}")
        return x      

In [14]:
class EncoderLayer(nn.Module):

    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x):
        residual_x = x   # 30*50*512
        print("------- ATTENTION 1 ------")
        x = self.attention(x, mask=None) # 30*50*512
        print("------- DROPOUT 1 ------")
        x = self.dropout1(x)  # 30*50*512
        print("------- ADD AND LAYER NORMALIZATION 1 ------")
        x = self.norm1(x + residual_x)  # 30*50*512
        residual_x = x
        print("------- ATTENTION 2 ------")
        x = self.ffn(x)   #30*50*512
        print("------- DROPOUT 2 ------")
        x = self.dropout2(x)   #30*50*512
        print("------- ADD AND LAYER NORMALIZATION 2 ------")
        x = self.norm2(x + residual_x)  #30*50*512
        return x

In [15]:
class Encoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers):
        super().__init__()
        self.layers = nn.Sequential(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                     for _ in range(num_layers)])

    def forward(self, x):
        x = self.layers(x)
        return x
     

In [16]:
d_model = 512
num_heads = 8
drop_prob = 0.1
batch_size = 30
max_sequence_length = 50
ffn_hidden = 2048
num_layers = 5

encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)

In [17]:
x = torch.randn( (batch_size, max_sequence_length, d_model) ) # includes positional encoding
out = encoder(x)

------- ATTENTION 1 ------
x.size(): torch.Size([30, 50, 512])
qkv.size(): torch.Size([30, 50, 1536])
qkv.size(): torch.Size([30, 50, 8, 192])
qkv.size(): torch.Size([30, 8, 50, 192])
q size: torch.Size([30, 8, 50, 64]), k size: torch.Size([30, 8, 50, 64]), v size: torch.Size([30, 8, 50, 64]), 
scaled.size() : torch.Size([30, 8, 50, 50])
values.size(): torch.Size([30, 8, 50, 64]), attention.size:torch.Size([30, 8, 50, 50]) 
values.size(): torch.Size([30, 50, 512])
out.size(): torch.Size([30, 50, 512])
------- DROPOUT 1 ------
------- ADD AND LAYER NORMALIZATION 1 ------
Mean (torch.Size([30, 50, 1]))
Standard Deviation  (torch.Size([30, 50, 1]))
y: torch.Size([30, 50, 512])
self.gamma: torch.Size([512]), self.beta: torch.Size([512])
out: torch.Size([30, 50, 512])
------- ATTENTION 2 ------
x after first linear layer: torch.Size([30, 50, 2048])
x after activation: torch.Size([30, 50, 2048])
x after dropout: torch.Size([30, 50, 2048])
x after 2nd linear layer: torch.Size([30, 50, 512])
-